# 1. 数据处理  pandas
- 合并表格
- 表格索引访问
- 数据转换
- 数据填充
# 2.分析
- 价格分布
- 评论分布
- 商品排行榜Top10
- 热词
# 3.可视化
- 价格分布  饼图
- 评分分布  饼图
- 商品排行Top10  直方图
- 热词  词云图

In [74]:
import pandas as pd
import numpy as np
import glob,os  # 分词  处理字符串拆分单词
from pyecharts import options as opts  # 配置项
from pyecharts.charts import Pie  # 饼图
from pyecharts import globals

class DDCharts():
    
    """初始化方法"""
    def __init__(self):
        # glob  根据指定的文件夹及匹配规则，匹配满足条件所有文件，返回文件列表
        # join: 关联路径
        self.files=glob.glob(os.path.join("./当当网商品数据/","第*页商品信息.csv"))
        self.data_pd=[]
        self.data_np=[]
        # print(self.files)

    """数据清洗"""
    def headleData(self):
        # 1.读取数据
        # 2.合并数据
        self.data_pd=pd.concat([pd.read_csv(file) for file in self.files])
        # print(self.data_pd.info())

        # 3.数据转换为map映射替换内容
        # unique():用于获取series对象的唯一值
        # print("提取唯一:",self.data_pd["商品评分"].unique())
        # 提取唯一: ['40' '10' '90' '60' '0' '0%' '80' '70' 0]
        maps={
            '10':1,
            '90':4.5,
            '0%':0,
            '80':4,
            '70':3.5,
            '0':0,
            '60':3
        }
        self.data_pd["商品评分new"]=self.data_pd["商品评分"].map(maps)
        # print(self.data_pd["商品评分new"].head())
        # 数据类型转换astype()
        #print(self.data_pd["商品价格"])
        #print("------------------------")
        self.data_pd["商品价格"]=self.data_pd["商品价格"].str.replace("¥",'').str.replace("起",'')
        self.data_pd["商品评论"]=self.data_pd["商品评论"].str.replace("条评论",'')
        self.data_pd["商品价格"]=self.data_pd["商品价格"].astype('float32')
        self.data_pd["商品评论"]=self.data_pd["商品评论"].astype('float32')
        # 商品详情
        #isnull:判断当前列是否存在null的数据，标记True
        self.data_pd["商品详情"].isnull()
        # fillna: 把当前列 被True标记的数据填充指定内容
        # self.data_pd["商品详情"].fillna("无内容",inplace=True)
        # df.fillna({ 'a': 100 }, inplace=True) 
        self.data_pd["商品详情"]=self.data_pd["商品详情"].fillna("无内容")
        pass

    # 绘图
    def parseData(self):
        # 1.numpy数据转化
        self.data_np=np.array(self.data_pd)
        # 2.价格分布
        prices=[]
        priceTop=['',0]
        pricedict={}# 饼图 {"价格区间"："数量"}
        for good in self.data_np:
            price=good[1]
            prices.append(price)
            if priceTop[1]<price: # 判断价格对比
                priceTop=[good[0],price]
        #print("价格最高商品",priceTop)

        # 分组统计
        pricedict["0-50元"]=np.sum([price>0 and price<50 for price in prices])
        pricedict["50-100元"]=np.sum([price>50 and price<100 for price in prices])
        pricedict["100-150元"]=np.sum([price>100 and price<150 for price in prices])
        pricedict["150-200元"]=np.sum([price>150 and price<200 for price in prices])
        pricedict["200元以上"]=np.sum([price>200 for price in prices])
        print("商品价格分布：",pricedict)
        # 绘图
        self.drawPie(pricedict,"商品价格分布图")
        pass

    # 价格分布图
    def drawPie(self,data,title):
        temp=[[key,int(value)] for key,value in data.items()]
        c = (
            Pie(init_opts=opts.InitOpts(
                    theme=globals.ThemeType.DARK
                )
            )
            .add(
                series_name=title,
                data_pair=temp,
                rosetype="radius", # 南丁格尔图，通过百分比和半径长度展示数据
                radius=["20%","80%"], # 中心
                center=["50%", "50%"], # 内圆与外圆的半径范围
            )
            # 全局配置项
            .set_global_opts(
                title_opts=opts.TitleOpts(  # 标题配置项
                    title=title,
                    pos_left="center", # 定位左居中
                    pos_top="20",  # 定位上
                    title_textstyle_opts=opts.TextStyleOpts(color="#fff"),
                ), 
                legend_opts=opts.LegendOpts(is_show=False), # 图例配置项，is_show=False是否显示
            )
            # 系列配置项
            .set_series_opts(
                  # 工具箱配置项
                   tooltip_opts=opts.TooltipOpts(
                        trigger="item", formatter="{a} <br/>{b}: {c} ({d}%)"
                    ),
                    # 标签配置项
                    label_opts=opts.LabelOpts(color="rgba(255, 255, 255, 0.3)"),
            )
        )
        c.render("./当当网商品数据/{}.html".format(title))
    
    """程序入口"""
    def main(self):
        self.headleData()
        # 可视化
        self.parseData()

# 启动
DDCharts().main()


商品价格分布： {'0-50元': 189, '50-100元': 136, '100-150元': 27, '150-200元': 5, '200元以上': 3}


In [41]:
pip install pyecharts -i https://pypi.tuna.tsinghua.edu.cn/simple

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ---------------------------------------- 0.0/149.4 kB ? eta -:--:--
     -- ------------------------------------- 10.2/149.4 kB ? eta -:--:--
     ------------------------ -------------- 92.2/149.4 kB 1.1 MB/s eta 0:00:01
     --------------------------- -------- 112.6/149.4 kB 939.4 kB/s eta 0:00:01
     ------------------------------------ 149.4/149.4 kB 809.1 kB/s eta 0:00:00
     ---------------------------------------- 0.0/75.6 kB ? eta -:--:--
     ---------------- ----------------------- 30.7/75.6 kB ? eta -:--:--
     ---------------- ----------------------- 30.7/75.6 kB ? eta -:--:--
     -------------------------------------- 75.6/75.6 kB 695.3 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.
